In [1]:
import os
def install_dependencies():
    !pip install -Uqq  git+https://github.com/huggingface/peft.git
    !pip install -Uqq transformers datasets accelerate bitsandbytes
    !pip install -Uqq wandb
    !pip install torch torchvision torchaudio|
    !pip install sentencepiece
# uncomment the following line to install the required dependencies
# install_dependencies()



In [2]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM
torch.cuda.is_available()


model_path = 'openlm-research/open_llama_3b_v2'




In [3]:
tokenizer = LlamaTokenizer.from_pretrained(model_path,add_eos_token=True)  


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [4]:
tokenizer.add_special_tokens({'eos_token':'<eos>'})
print('eos_token_id:',tokenizer.eos_token_id)

eos_token_id: 32000


In [5]:
def tokenize(prompt, tokenizer):
    result= tokenizer(
        prompt+"<eos>",  # add the end-of-stream token
        padding="max_length",
        max_length=2048,
        truncation=True
    )

    return {"input_ids": result["input_ids"], "attention_mask": result["attention_mask"]}
    

In [6]:
from datasets import load_dataset
data = load_dataset("csv", data_files= "C:/Users/Kevin Zhang/Documents/GitHub/dafny/dataset.csv", split='train[:400]')

In [7]:
data

Dataset({
    features: ['incorrect_program', 'verifier_output', 'correct_program'],
    num_rows: 400
})

In [8]:
train_val = data.train_test_split(test_size=len(data)//5, shuffle=True, seed=42)

In [9]:
train_data = train_val["train"]
test_data = train_val["test"]

In [10]:
def generate_prompt(data_point): 
    return f"""Below is an incorrect Dafny program and the corresponding error output from the Dafny verifier. Assume that the program specification is correct, and the program implementation is wrong. Correct the program so that is satisfies the specification and fixes the verifier error.
    
### Incorrect Dafny Program:
{data_point["incorrect_program"]}
    
### Error output from the verifier:
{data_point["verifier_output"]}

### Corrected program:
{data_point["correct_program"]}"""

In [11]:
print(generate_prompt(data[5]))

Below is an incorrect Dafny program and the corresponding error output from the Dafny verifier. Assume that the program specification is correct, and the program implementation is wrong. Correct the program so that is satisfies the specification and fixes the verifier error.
    
### Incorrect Dafny Program:
include "../Wrappers.dfy"
include "../Functions.dfy"
include "../Collections/Sequences/Seq.dfy"
include "Unicode.dfy"
// UnicodeEncodingForm.dfy


abstract module {:options "-functionSyntax:4"} UnicodeEncodingForm {
  function IsMinimalWellFormedCodeUnitSubsequence(s: CodeUnitSeq): (b: bool)
    ensures b ==> |s| > 0 && forall i | 0 < i < |s| :: !IsMinimalWellFormedCodeUnitSubsequence(s[..i])
    decreases |s|

  function SplitPrefixMinimalWellFormedCodeUnitSubsequence(s: CodeUnitSeq): (maybePrefix: Option<MinimalWellFormedCodeUnitSeq>)
    ensures |s| == 0 ==> maybePrefix.None?
    ensures (exists i | 0 < i <= |s| :: IsMinimalWellFormedCodeUnitSubsequence(s[..i])) <==> true && may

In [12]:
train_data = train_data.map(lambda x: ({"text": generate_prompt(x) +"<eos>" }))
test_data = test_data.map(lambda x: ({"text": generate_prompt(x) +"<eos>"}))




In [13]:
from transformers import AutoModelForCausalLM
import bitsandbytes as bnb
from transformers import BitsAndBytesConfig
config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_use_double_quant=True, bnb_4bit_compute_dtype=torch.bfloat16)
model = LlamaForCausalLM.from_pretrained(
    model_path,
    device_map={"":0},
    quantization_config=config
)

bin c:\users\kevin zhang\appdata\local\programs\python\python39\lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


In [14]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
print(model)
lora_config = LoraConfig(
 r= 8, 
 lora_alpha=8,
 target_modules=[ "q_proj",
    "o_proj",
    "k_proj",
    "v_proj",
    "gate_proj",
    "down_proj",
    "up_proj",
    "lm_head"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.CAUSAL_LM
)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 3200, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=3200, out_features=3200, bias=False)
          (k_proj): Linear4bit(in_features=3200, out_features=3200, bias=False)
          (v_proj): Linear4bit(in_features=3200, out_features=3200, bias=False)
          (o_proj): Linear4bit(in_features=3200, out_features=3200, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3200, out_features=8640, bias=False)
          (up_proj): Linear4bit(in_features=3200, out_features=8640, bias=False)
          (down_proj): Linear4bit(in_features=8640, out_features=3200, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
   

In [15]:
model = prepare_model_for_kbit_training(model)

# add LoRA adaptor


In [16]:

from trl import SFTTrainer
import transformers
eval_steps = 10
save_steps = 10
logging_steps = 10
output_dir = "C:/Users/Kevin Zhang/Documents/GitHub/dafny/trainingout"

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="text",
    args=transformers.TrainingArguments(
        num_train_epochs=1,
        learning_rate=3e-4,
        logging_steps=logging_steps,
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=eval_steps,
        save_steps=save_steps,
        output_dir=output_dir,
        save_total_limit=3,
        load_best_model_at_end=True,
        push_to_hub=False,
        auto_find_batch_size=True
    ),
    peft_config=lora_config
)

c:\users\kevin zhang\appdata\local\programs\python\python39\lib\site-packages\trl\trainer\ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(
c:\users\kevin zhang\appdata\local\programs\python\python39\lib\site-packages\trl\trainer\sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/320 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

c:\users\kevin zhang\appdata\local\programs\python\python39\lib\site-packages\trl\trainer\sft_trainer.py:247: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [17]:
trainer.train()


wandb: Currently logged in as: klebn. Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
c:\users\kevin zhang\appdata\local\programs\python\python39\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
10,0.719900,0.509201
20,0.406600,0.330660
30,0.242700,0.257715
40,0.219400,0.236480


c:\users\kevin zhang\appdata\local\programs\python\python39\lib\site-packages\peft\utils\save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
c:\users\kevin zhang\appdata\local\programs\python\python39\lib\site-packages\peft\utils\save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
c:\users\kevin zhang\appdata\local\programs\python\python39\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is reco

TrainOutput(global_step=40, training_loss=0.3971314251422882, metrics={'train_runtime': 54477.2952, 'train_samples_per_second': 0.006, 'train_steps_per_second': 0.001, 'total_flos': 6560942968012800.0, 'train_loss': 0.3971314251422882, 'epoch': 1.0})

In [18]:
trainer.save_model("openllama-3b-int4-dafny-3")

In [19]:
trainer.tokenizer.save_pretrained("openllama-3b-int4-dafny-3")

('openllama-3b-int4-dafny-3\\tokenizer_config.json',
 'openllama-3b-int4-dafny-3\\special_tokens_map.json',
 'openllama-3b-int4-dafny-3\\tokenizer.model',
 'openllama-3b-int4-dafny-3\\added_tokens.json',
 'openllama-3b-int4-dafny-3\\tokenizer.json')

In [20]:
trainer.model.config.to_json_file("openllama-3b-int4-dafny-3/config.json")
